In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
from difflib import SequenceMatcher


In [2]:
query = "iphone"

query = query.lower().replace(" ", "-")

url = f"https://www.elcorteingles.es/search/?s={query}"
data = rq.get(url)
print(data.text)


<!DOCTYPE html><html lang="es"><head><link rel="preconnect" href="//cuenta.elcorteingles.es" /><link rel="preconnect" href="//api.elcorteingles.es" /><link rel="preconnect" href="//sgfm.elcorteingles.es" /><link rel="preconnect" href="//www.googletagmanager.com" /><link rel="preconnect" href="//www.google-analytics.com" /><link rel="preconnect" href="//recs.richrelevance.com" /><link rel="prefetch" href="/api/stock?products=A20182660%2CA20182685%2CA20182729%2CA27722117%2CA20182667%2CA27722116%2CA27721746%2CA27719627%2CA24035210%2CA27721765%2CA27721763%2CA27722127%2CA27719619%2CA24366949%2CA27721773%2CA27722115%2CA20182688%2CA24035216%2CA27722311%2CA24035226%2CA27723923%2CA24366947%2CA27719647%2CA24035214" /><title>Móviles y Smartphones libres | El Corte Inglés</title><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no" /><meta name="generator" content="Moonshine" /><me

In [3]:
query = "iphone"

query = query.lower().replace(" ", "-")

url = f"https://www.elcorteingles.es/search/??itemsPerPage=36&s={query}"
data = rq.get(url)

soup_eci = BeautifulSoup(data.text, 'html.parser')

names_eci = soup_eci.select("a.js-product-click", recursive=False)
links_eci = soup_eci.select("a.js-product-click", recursive=False)
prices_eci = soup_eci.select("div.product-price span.current", recursive=False)
old_prices_eci = soup_eci.select("div.info", recursive=False)
images_eci = soup_eci.select(".product-preview img", recursive=False)

names_eci = [e["title"] for e in names_eci]
links_eci = [e["href"] for e in links_eci]
prices_eci = [e.text[:-1] for e in prices_eci]
old_prices_eci = [e.select("span.stroked")[0].text[:-1] if e.select("span.stroked") else "" for e in old_prices_eci]
images_eci = [e["src"] for e in images_eci]

print(names_eci, len(names_eci))
print(links_eci, len(links_eci))
print(prices_eci, len(prices_eci))
print(old_prices_eci, len(old_prices_eci))
print(images_eci, len(images_eci))

['iPhone 6s 32GB Gris espacial móvil libre', 'iPhone 7 32GB Negro móvil libre', 'iPhone 7 Plus 32GB Negro móvil libre', 'iPhone XR 64GB Negro móvil libre', 'iPhone 6s 32GB Oro rosa móvil libre', 'iPhone XR 64GB Blanco móvil libre', 'iPhone XS Max 64GB Plata móvil libre', 'iPhone XS 64GB Gris espacial móvil libre', 'iPhone 8 64 GB Gris Espacial móvil libre', 'iPhone XS Max 256GB Plata móvil libre', 'iPhone XS Max 256GB Oro móvil libre', 'iPhone XR 128GB Negro móvil libre', 'iPhone XS 64GB Plata móvil libre', 'iPhone X 64 GB Gris espacial móvil libre', 'iPhone XS Max 256GB Gris espacial móvil libre', 'iPhone XR 64GB (PRODUCT)RED móvil libre', 'iPhone 7 32GB Plata móvil libre', 'iPhone 8 64 GB Oro móvil libre', 'iPhone XR 64GB Azul móvil libre', 'iPhone 8 Plus 64 GB Oro móvil libre', 'iPhone XS Max 512GB Oro móvil libre', 'iPhone X 64 GB Plata móvil libre', 'iPhone XS 256GB Oro móvil libre', 'iPhone 8 64 GB Plata móvil libre'] 24
['/electronica/A20182660-iphone-6s-32gb-gris-espacial-movil

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import os

class PageGetter:
    driver = False
    def __init__(self, defaultBrowser='firefox'):
        if defaultBrowser == 'firefox':
            # Se configura un perfil con una carpeta de descarga para Selenium
            self.profile = webdriver.FirefoxProfile()
            self.profile.set_preference("browser.download.folderList", 2)
            self.profile.set_preference("browser.download.manager.showWhenStarting", False)
            '''# Carpeta destino:
            self.profile.set_preference("browser.download.dir", os.path.dirname(os.path.realpath(__file__))+"/download-data/")
            # Se le dice a Selenium que no pregunte cuando se descarge .zips
            self.profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/zip")
'''
            self.driver = webdriver.Firefox(self.profile)
        elif defaultBrowser == 'chrome':
            self.driver = webdriver.Chrome()
        else:
            raise Exception('Not a valid browser')
    
    # La función getPage recorre los elementos de la página de la CNMV y va pinchando en donde nos interesa.
    # He puesto algunos time.sleep() porque cuando la conexión es lenta, Selenium hace click antes de que el elemento se cargue y pueda haber error.

    def getPage(self, url):
        if not self.driver:
            raise Exception("You should start a browser connection")
        print ("Scrapeando GearBest...")
        self.driver.get(url)
        gb_name = self.driver.find_elements_by_class_name('gbGoodsItem_title')
        gb_price = self.driver.find_elements_by_class_name('gbGoodsItem_price')
        gb_image = self.driver.find_elements_by_class_name('gbGoodsItem_image')
        
        gb_names = [e.get_attribute("innerText") for e in gb_name]
        gb_prices = [e.get_attribute("innerText")[:-1] for e in gb_price]
        gb_images = [e.get_attribute("src") for e in gb_image]

        return gb_names, gb_prices, gb_images
        # time.sleep(2)
        
    
    def close(self):
        self.driver.quit()
        
query = "xiaomi redmi"

query = query.lower().replace(" ", "-")

url = f"https://es.gearbest.com/{query}-_gear/"

pg = PageGetter('firefox')
gb_names, gb_prices, gb_images = pg.getPage(url)
pg.close()
print(gb_names, gb_prices, gb_images)
'''
soup_eci = BeautifulSoup(data.text, 'html.parser')

names = soup_eci.select("a.js-product-click", recursive=False)
prices = soup_eci.select("div.product-price span.current", recursive=False)
images = soup_eci.select("img", recursive=False)

names = [e["title"] for e in names]
prices = [e.text[:-1] for e in prices]
images = [e["src"] for e in images]

print(names)
print(prices)
print(images)'''

Scrapeando GearBest...
['ASLING Protector Transparente Forro de Teléfono de TPU para Xiaomi Redmi 4A', 'NILLKIN Funda Protectora de Inversión de Función de Sueño para XIAOMI Redmi Note 3', 'ASLING Película de Pantalla de Vidrio Templado para Xiaomi Redmi S2 de 2 Piezas', 'Xiaomi Redmi K20 Phablet 4G', 'Xiaomi Redmi Note 7 Pro Phablet 4G', 'Xiaomi Redmi Note 7 Phablet 4G de Versión Global', 'Xiaomi Redmi Note 7 Phablet 4G de Versión Global 4GB RAM', 'Xiaomi Redmi K20 Pro Phablet 4G Cámara Trasera de 48,0MP', 'Original Carcasa Trasera Reemplazable para XiaoMi RedMi Note 2', 'Xiaomi Redmi Note 7 Phablet 4G 3GB RAM de Versión Global', 'Xiaomi Redmi K20 Phablet 4G 8GB RAM 256GB ROM', 'Caso Yeshold para Xiaomi Redmi Nota 3', 'Luanke Cubierta Protectora Transparente para Xiaomi Redmi 3 Pro', 'ASLING Funda de Teléfono Transparente para Xiaomi Redmi Note 4', 'Cubierta Transparente Suave de TPU para Xiaomi Redmi Note 4', 'Xiaomi Redmi K20 Pro 4G Phablet Snapdragon 855', 'Luanke Funda Protector T

'\nsoup_eci = BeautifulSoup(data.text, \'html.parser\')\n\nnames = soup_eci.select("a.js-product-click", recursive=False)\nprices = soup_eci.select("div.product-price span.current", recursive=False)\nimages = soup_eci.select("img", recursive=False)\n\nnames = [e["title"] for e in names]\nprices = [e.text[:-1] for e in prices]\nimages = [e["src"] for e in images]\n\nprint(names)\nprint(prices)\nprint(images)'

In [5]:
import re
query = "iphone"

query = query.lower().replace(" ", "-")

url = f"https://www.mediamarkt.es/es/search.html?query={query}&searchProfile=onlineshop"
data_mm = rq.get(url)

soup_mm = BeautifulSoup(data_mm.text, 'html.parser')

names_mm = soup_mm.select("span.photo", recursive=False)
links_mm = soup_mm.select("aside.product-photo span.photo", recursive=False)
prices_mm = soup_mm.select("div.price-box", recursive=False)
old_prices_mm = soup_mm.select("div.info span.stroked", recursive=False)
images_mm = soup_mm.select("span.photo", recursive=False)

names_mm = [e.select("img", recursive=False)[0]["alt"] for e in names_mm]
links_mm = [e["data-clickable-href"] for e in links_mm]
prices_mm = [re.findall("^\d+[.,]\d+|^\d+",e.text[1:])[0] for e in prices_mm]
old_prices_mm = [""]*len(names_mm)
images_mm = [e.select("img", recursive=False)[0]["data-original"] for e in images_mm]

print(names_mm, len(names_mm))
print(links_mm, len(links_mm))
print(prices_mm, len(prices_mm))

print(images_mm, len(images_mm))

['Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Rojo', 'Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Rojo', 'Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Negro', 'Apple iPhone 6S Plus, 5.5", 32GB, Red 4G, Oro rosa', 'Apple iPhone 8, 64GB, Red 4G LTE, Pantalla Retina HD de 4.7", Gris espacial', 'Apple iPhone 7 PLUS de 32GB, pantalla Retina HD de 5.5" y doble , Dorado', 'Apple iPhone XS, 64GB, OLED SuperRetinaHD 5.8”, Chip A12 Bionic, DualCam, FaceID, IP68, Gris', 'Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Azul', 'Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Coral', 'Apple iPhone 8, 64GB, Red 4G LTE, Pantalla Retina HD de 4.7", Dorado', 'Apple iPhone 8 Plus, 5.5" Full HD, 64 GB, Oro', 'Apple iPhone Xs, 256 GB, 5.8" OLED Super Retina HD, A12 Bionic, Plata', 'Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Blanco', 'Apple iPhone XR, 64 GB, Liqui

In [6]:
query = "iphone"

query = query.lower().replace(" ", "-")

url = f"https://www.carrefour.es/global/Ntt=pollo&search=Buscar"
data_prueba = rq.get(url)

soup_prueba = BeautifulSoup(data_prueba.text, 'html.parser')

print(soup_prueba)


<!DOCTYPE html>

<html lang="es">
<head>
<script src="//static.carrefour.es/store-app/v2603/js/tealeaf-w3c-prod-min-5.2.0.js" type="text/javascript"></script>
<script src="https://www.carrefour.es/_includes/js/cookies2018min.js" type="text/javascript"></script>
<script type="text/javascript">
  var pageModel = {
    config : {
  "repeat_last_order_supermarket" : "supermercado/PedidosAnterioresyListas",
  "tealeafScriptPath" : "/js/tealeaf-w3c-prod-min-5.2.0.js",
  "baseCellarUrl" : "https://www.carrefour.es/bodega",
  "analytics_enabled" : true,
  "locale" : "es_ES",
  "url_nonfood_cart" : "shopping_cart",
  "salePoint" : "004320",
  "url_order_confirmation" : "confirmation",
  "urlServicesNf" : "/api/unified_menu/",
  "baseUrl" : "https://www.carrefour.es/",
  "wish_list_supermarket" : "supermercado/PedidosAnterioresyListas",
  "DataLayer" : "{\"pageEnvironment\":\"pro\",\"pageVersionValue\":\"nn\",\"pageCategoryExtra\":\"nn\",\"pageEnvironmentDetail\":\"Web\",\"pageSiteName\":\"tien

In [7]:
query = "iphone"

query = query.lower().replace(" ", "-")

url = f"https://www.fnac.es/SearchResult/ResultList.aspx?ItemPerPage=100&SDM=list&Search={query}"
data_fnac = rq.get(url)

soup_fnac = BeautifulSoup(data_fnac.text, 'html.parser')

names_fnac = soup_fnac.select("div.Article-itemVisual img", recursive=False)
links_fnac = soup_fnac.select("a.Article-title", recursive=False)
prices_fnac = soup_fnac.select(".Article-price", recursive=False)
old_prices_fnac = soup_fnac.select(".Article-price", recursive=False)
images_fnac = soup_fnac.select("div.Article-itemVisual img", recursive=False)

names_fnac = [e["alt"] for e in names_fnac]
links_fnac = [e["href"] for e in links_fnac]
prices_fnac = [re.findall("\d+[.,]\d+|\d+",e.select(".userPrice", recursive=False)[0].text)[0] for e in prices_fnac]

old_prices_fnac = [re.findall("\d+[.,]\d+|\d+",e.select(".oldPrice", recursive=False)[0].text)[0] if e.select(".oldPrice", recursive=False) else "" for e in old_prices_fnac]

images_fnac = [e["src"] for e in images_fnac]

print(names_fnac, len(names_fnac))
print(links_fnac, len(links_fnac))
print(prices_fnac, len(prices_fnac))
print(old_prices_fnac, len(old_prices_fnac))
print(images_fnac, len(images_fnac))

['Apple iPhone 7 128 GB Oro rosa', 'Apple iPhone 8 256GB Oro', 'Apple iPhone 7 Plus 128GB negro', 'Apple iPhone 7 128 GB negro', 'Apple iPhone 6s 64 GB gris espacial', 'Apple iPhone Xr 128GB Negro', 'Apple iPhone Xr 128GB Blanco', 'Apple iPhone 8 Plus 64GB Gris espacial', 'Apple iPhone Xr 128GB Azul', 'Apple iPhone X 64GB Gris espacial', 'Apple iPhone Xr 128GB Amarillo', 'Apple iPhone 6s - plata - 4G LTE, LTE Advanced - 64 GB - TD-SCDMA / UMTS / GSM - teléfono inteligente', 'Apple iPhone Xr 64GB Negro', 'Apple iPhone 7 32GB Negro mate', 'Apple Iphone 7 128 GB Plata - Smartphone Libre de 4,7 pulgadas', 'Apple iPhone 7 32GB Negro mate (Producto Reacondicionado)', 'Apple iPhone 8 256GB (PRODUCT)RED', 'Apple iPhone Xr 128GB Coral', 'Apple iPhone Xr 64GB Blanco', 'Apple iPhone Xs 256GB Gris Espacial'] 20
['https://www.fnac.es/Apple-iPhone-7-128-GB-Oro-rosa-Telefono-movil-Smartphone/a1290724#omnsearchpos=1', 'https://www.fnac.es/Apple-iPhone-8-256GB-Oro-Telefono-movil-Smartphone/a1403915#omn

In [8]:

query = "iphone"

query = query.lower().replace(" ", "-")

url = f"https://www.worten.es/search?sortBy=relevance&hitsPerPage=48&page=1&query={query}"
data_wrtn = rq.get(url)

soup_wrtn = BeautifulSoup(data_wrtn.text, 'html.parser')

names_wrtn = soup_wrtn.select("h3.w-product__title", recursive=False)
links_wrtn = soup_wrtn.select("a.w-product__url", recursive=False)
prices_wrtn = soup_wrtn.select("span.w-currentPrice", recursive=False)
old_prices_wrtn = soup_wrtn.select("div.bigPricerFA", recursive=False)
images_wrtn = soup_wrtn.select("figure.w-product__image img", recursive=False)

names_wrtn = [e.text for e in names_wrtn]
links_wrtn = [e["href"] for e in links_wrtn]
prices_wrtn = [re.findall("\d+[.,]\d+|\d+",e.text)[0] for e in prices_wrtn]
old_prices_wrtn = [""]*len(prices_wrtn)
images_wrtn = [e["data-src"] for e in images_wrtn]

print(names_wrtn, len(names_wrtn))
print(links_wrtn, len(links_wrtn))
print(prices_wrtn, len(prices_wrtn))
print(old_prices_wrtn, len(old_prices_wrtn))
print(images_wrtn, len(images_wrtn))




['Protector de cristal templado SBS Ultra iPhone 6, 6s, 7, 8', "iPhone\xa07 APPLE (4.7'' - 2 GB - 32 GB - Negro)", "iPhone 6s Plus APPLE (5.5'' - 2 GB - 32 GB - Gris)", "iPhone XS APPLE (5.8'' - 4 GB - 256 GB - Plateado)", "iPhone\xa07 APPLE (4.7'' - 2 GB - 128 GB - Dorado)", "iPhone XS APPLE (5.8'' - 4 GB - 64 GB - Gris Espacial)", "iPhone XS Max APPLE (6.5'' - 4 GB - 512 GB - Dorado)", 'Carcasa SBS Slim iPhone 6, 6s negro', "iPhone XR APPLE (6.1'' - 3 GB - 64 GB - Negro)", "iPhone XR APPLE (6.1'' - 3 GB - 64 GB - Negro)", "iPhone XS Max APPLE (6.5'' - 4 GB - 256 GB - Plateado)", "iPhone\xa07 Plus APPLE (5.5'' - 3 GB - 32 GB - Negro)", "iPhone XS APPLE (5.8'' - 4 GB - 64 GB - Plata)", "iPhone XR APPLE (6.1'' - 3 GB - 64 GB - Azul)", "iPhone XS Max APPLE (6.5'' - 4 GB - 64 GB - Dorado)", "iPhone XR APPLE (6.1'' - 3 GB - 128 GB - Amarillo)", "iPhone XS Max APPLE (6.5'' - 4 GB - 256 GB - Dorado)", "iPhone\xa07 Plus APPLE (5.5'' - 3 GB - 32 GB - Rosa)", "iPhone XS Max APPLE (6.5'' - 4 GB 

In [9]:
names = names_eci + names_mm + names_fnac + names_wrtn
links = links_eci + links_mm + links_fnac + links_wrtn
prices = prices_eci + prices_mm + prices_fnac + prices_wrtn
old_prices = old_prices_eci + old_prices_mm + old_prices_fnac + old_prices_wrtn
images = images_eci + images_mm + images_fnac + images_wrtn
shop = ["ECI"]*len(names_eci) + ["MM"]*len(names_mm) + ["FNAC"]*len(names_fnac) + ["WORTEN"]*len(names_wrtn)

print(len(names), len(links), len(prices), len(old_prices), len(images), len(shop))
dic_shops = {
    "products": names,
    "links": links,
    "prices": prices,
    "oldPrices": old_prices,
    "images": images,
    "shop": shop
}


results = pd.DataFrame(dic_shops)

display(results)

112 112 112 112 112 112


,products,links,prices,oldPrices,images,shop
0,iPhone 6s 32GB Gris espacial móvil libre,/electronica/A20182660-iphone-6s-32gb-gris-esp...,409,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2019...,ECI
1,iPhone 7 32GB Negro móvil libre,/electronica/A20182685-iphone-7-32gb-negro-mov...,529,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2016...,ECI
2,iPhone 7 Plus 32GB Negro móvil libre,/electronica/A20182729-iphone-7-plus-32gb-negr...,529,659,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2016...,ECI
3,iPhone XR 64GB Negro móvil libre,/electronica/A27722117-iphone-xr-64gb-negro-mo...,859,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2018...,ECI
4,iPhone 6s 32GB Oro rosa móvil libre,/electronica/A20182667-iphone-6s-32gb-oro-rosa...,409,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2019...,ECI
5,iPhone XR 64GB Blanco móvil libre,/electronica/A27722116-iphone-xr-64gb-blanco-m...,859,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2018...,ECI
6,iPhone XS Max 64GB Plata móvil libre,/electronica/A27721746-iphone-xs-max-64gb-plat...,1.259,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2019...,ECI
7,iPhone XS 64GB Gris espacial móvil libre,/electronica/A27719627-iphone-xs-64gb-gris-esp...,1.159,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2019...,ECI
8,iPhone 8 64 GB Gris Espacial móvil libre,/electronica/A24035210-iphone-8-64-gb-gris-esp...,689,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2019...,ECI
9,iPhone XS Max 256GB Plata móvil libre,/electronica/A27721765-iphone-xs-max-256gb-pla...,1.429,,//sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/2019...,ECI


In [10]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [138]:
import textdistance
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [142]:
i = "MM"
results[results["shop"]==i]["products"].to_list()

['Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Rojo',
 'Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Rojo',
 'Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Negro',
 'Apple iPhone 6S Plus, 5.5", 32GB, Red 4G, Oro rosa',
 'Apple iPhone 8, 64GB, Red 4G LTE, Pantalla Retina HD de 4.7", Gris espacial',
 'Apple iPhone 7 PLUS de 32GB, pantalla Retina HD de 5.5" y doble , Dorado',
 'Apple iPhone XS, 64GB, OLED SuperRetinaHD 5.8”, Chip A12 Bionic, DualCam, FaceID, IP68, Gris',
 'Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Azul',
 'Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Coral',
 'Apple iPhone 8, 64GB, Red 4G LTE, Pantalla Retina HD de 4.7", Dorado',
 'Apple iPhone 8 Plus, 5.5" Full HD, 64 GB, Oro',
 'Apple iPhone Xs, 256 GB, 5.8" OLED Super Retina HD, A12 Bionic, Plata',
 'Apple iPhone XR, 128 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Blanco',
 'Apple iPhone XR,

In [177]:
'''tupla_elementos = []
for e in range(len(results["products"])):
    tupla_elementos.append((e, results["products"][e]))

print(tupla_elementos)
'''
founded = []
similares = []
for e in range(len(results["products"])):
    shops = results[results["shop"]!=results["shop"][e]].copy()
    fila = [(results["products"][e],100)]
    for i in shops["shop"].unique():
        distance = max(process.extract(results["shop"][e], results[results["shop"]==i]["products"].to_list(),\
                                         scorer=__custom_jaccard_similarity), key=lambda x:x[1])
        if distance[1] > .8:
            fila.append(distance)
    similares.append(fila)
    
'''similarity_matrix = []
founded = []
for i in range(len(dic_shops["products"])):
    similarity_matrix.append([])
    for e in range(len(dic_shops["products"])):
        if (i != e) & (results["shop"][i] != results["shop"][e]) & (results["shop"][e].index not in founded) :
            ratio = fuzz.partial_ratio(dic_shops["products"][i],dic_shops["products"][e])
            if ratio >= 50:
                similarity_matrix[i].append((ratio,e))
                for k in similarity_matrix[i]:
                    founded.append(k[1])'''

'''similares = []
founded = []
for e in range(len(similarity_matrix)):
    similares.append([i for i in sorted(similarity_matrix[e], reverse = True) if i[0] >= 50 and i[1] not in founded])
    for e in similares[e]:
        founded.append(e[1])

similares = [x for x in similares if x != []]'''
print(similares)

[[('iPhone 6s 32GB Gris espacial móvil libre', 100)], [('iPhone 7 32GB Negro móvil libre', 100)], [('iPhone 7 Plus 32GB Negro móvil libre', 100)], [('iPhone XR 64GB Negro móvil libre', 100)], [('iPhone 6s 32GB Oro rosa móvil libre', 100)], [('iPhone XR 64GB Blanco móvil libre', 100)], [('iPhone XS Max 64GB Plata móvil libre', 100)], [('iPhone XS 64GB Gris espacial móvil libre', 100)], [('iPhone 8 64 GB Gris Espacial móvil libre', 100)], [('iPhone XS Max 256GB Plata móvil libre', 100)], [('iPhone XS Max 256GB Oro móvil libre', 100)], [('iPhone XR 128GB Negro móvil libre', 100)], [('iPhone XS 64GB Plata móvil libre', 100)], [('iPhone X 64 GB Gris espacial móvil libre', 100)], [('iPhone XS Max 256GB Gris espacial móvil libre', 100)], [('iPhone XR 64GB (PRODUCT)RED móvil libre', 100)], [('iPhone 7 32GB Plata móvil libre', 100)], [('iPhone 8 64 GB Oro móvil libre', 100)], [('iPhone XR 64GB Azul móvil libre', 100)], [('iPhone 8 Plus 64 GB Oro móvil libre', 100)], [('iPhone XS Max 512GB Oro m

In [61]:
for i in range(1,2):
    for e in similares[i]:
        print(results.iloc[[e[1]]])

                                             products  \
59  Apple iPhone 7 32GB Negro mate (Producto Reaco...   

                                                links  prices oldPrices  \
59  https://www.fnac.es/Apple-iPhone-7-32GB-Negro-...  349,90             

                                               images  shop  
59  data:image/svg+xml,%3Csvg xmlns='http://www.w3...  FNAC  
                       products  \
49  Apple iPhone Xr 128GB Negro   

                                                links prices oldPrices  \
49  https://www.fnac.es/Apple-iPhone-Xr-128GB-Negr...    859       919   

                                               images  shop  
49  data:image/svg+xml,%3Csvg xmlns='http://www.w3...  FNAC  
                       products  \
47  Apple iPhone 7 128 GB negro   

                                                links  prices oldPrices  \
47  https://www.fnac.es/Apple-iPhone-7-128-GB-negr...  419,90    449,90   

                                            

In [95]:
import nltk

ModuleNotFoundError: No module named 'nltk'

In [115]:
fuzz.token_set_ratio('Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Negro', \
'iPhone XR 64GB Negro móvil libre')

65

In [116]:
fuzz.token_set_ratio('Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Rojo', \
'iPhone XR 64GB Negro móvil libre')

47

In [117]:
fuzz.token_set_ratio('Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Negro', \
'iPhone XR 128GB Negro móvil libre')

64

In [118]:
fuzz.token_set_ratio('Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Negro', \
'iPhone Xs 64GB Negro móvil libre')

56

In [80]:
def __custom_jaccard_similarity(x, y):

    setX = set(x)
    setY = set(y)

    intersection_cardinality = len(set.intersection(*[setX, setY]))
    union_cardinality = len(setX) + len(setY) - intersection_cardinality
    return intersection_cardinality/float(union_cardinality)

In [ ]:
'Apple iPhone XR, 64 GB, Liquid Retina HD 6.1”, A12 Bionic, 4K, IP67, Rojo',
'Apple iPhone Xr 64GB Blanco', 
"iPhone\xa07 APPLE (4.7'' - 2 GB - 32 GB - Negro)"
'iPhone 6s 32GB Gris espacial móvil libre'